In [23]:
import pandas as pd
import geopandas as gpd

In [30]:
df = pd.read_csv('data/berlin_temp_clean/berlin_all_districts_temp.csv', index_col=0)
df.head()

,SOUID,DATE,TG,Q_TG,Station,Station_name
0,127488,18760101,22,0,4005,BERLIN-TEGEL
1,127488,18760102,25,0,4005,BERLIN-TEGEL
2,127488,18760103,3,0,4005,BERLIN-TEGEL
3,127488,18760104,-58,0,4005,BERLIN-TEGEL
4,127488,18760105,-98,0,4005,BERLIN-TEGEL


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712684 entries, 0 to 712683
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0    SOUID        712684 non-null  int64 
 1       DATE      712684 non-null  int64 
 2      TG         712684 non-null  int64 
 3    Q_TG         712684 non-null  int64 
 4   Station       712684 non-null  int64 
 5   Station_name  712684 non-null  object
dtypes: int64(5), object(1)
memory usage: 38.1+ MB


In [38]:
# remove whitespaces in df
df = df.rename(columns=lambda x: x.strip())

In [39]:
# convert the temperature into unit of 1°C 
df['TG'] = df['TG']/10

In [40]:
df['datetime'] = pd.to_datetime(df['DATE'], format='%Y%m%d')
df['year'] = pd.DatetimeIndex(df['datetime']).year
df['month'] = pd.DatetimeIndex(df['datetime']).month

df.head()

,SOUID,DATE,TG,Q_TG,Station,Station_name,datetime,year,month
0,127488,18760101,2.2,0,4005,BERLIN-TEGEL,1876-01-01,1876,1
1,127488,18760102,2.5,0,4005,BERLIN-TEGEL,1876-01-02,1876,1
2,127488,18760103,0.3,0,4005,BERLIN-TEGEL,1876-01-03,1876,1
3,127488,18760104,-5.8,0,4005,BERLIN-TEGEL,1876-01-04,1876,1
4,127488,18760105,-9.8,0,4005,BERLIN-TEGEL,1876-01-05,1876,1


In [ ]:
# align the weather station names in the dataframe with the district names in the shapefile 
df['Station_name']







In [45]:
#Read in the shapefile with geopandas
SHAPEFILE = 'data/shapefile.shp/lor_ortsteile.shp'
gdf = gpd.read_file(SHAPEFILE) #[['ADMIN', 'geometry']]
gdf

,gml_id,spatial_na,spatial_al,spatial_ty,OTEIL,BEZIRK,FLAECHE_HA,geometry
0,re_ortsteil.0101,0101,Mitte,Polygon,Mitte,Mitte,1063.8748,"POLYGON ((13.41649 52.52696, 13.41669 52.52688..."
1,re_ortsteil.0102,0102,Moabit,Polygon,Moabit,Mitte,768.7909,"POLYGON ((13.33884 52.51974, 13.33879 52.51969..."
2,re_ortsteil.0103,0103,Hansaviertel,Polygon,Hansaviertel,Mitte,52.5337,"POLYGON ((13.34322 52.51557, 13.34319 52.51556..."
3,re_ortsteil.0104,0104,Tiergarten,Polygon,Tiergarten,Mitte,516.0672,"POLYGON ((13.36879 52.49878, 13.36876 52.49879..."
4,re_ortsteil.0105,0105,Wedding,Polygon,Wedding,Mitte,919.9112,"POLYGON ((13.34656 52.53879, 13.34648 52.53880..."
...,...,...,...,...,...,...,...,...
91,re_ortsteil.1207,1207,Waidmannslust,Polygon,Waidmannslust,Reinickendorf,223.7780,"POLYGON ((13.33106 52.61487, 13.33156 52.61449..."
92,re_ortsteil.1208,1208,Lübars,Polygon,Lübars,Reinickendorf,499.1961,"POLYGON ((13.36693 52.62535, 13.36698 52.62537..."
93,re_ortsteil.1209,1209,Wittenau,Polygon,Wittenau,Reinickendorf,587.1148,"POLYGON ((13.34413 52.58284, 13.34518 52.58148..."
94,re_ortsteil.1210,1210,Märkisches Viertel,Polygon,Märkisches Viertel,Reinickendorf,323.9201,"POLYGON ((13.34756 52.59008, 13.34754 52.59009..."


In [44]:
# groupby Station and year
df = df.groupby(['Station','year'])[['TG']].mean().reset_index()
df.head()
df.shape

(1962, 3)

In [ ]:

# merge geodataframa with dataframe
gdf = gdf.merge(df, left_on='ADMIN', right_on='country')
gdf.drop('ADMIN', axis=1, inplace=True)

# turn it into a geojson 
gdf.to_json()

# generate a GEOJSON string for a single year
gdf_2000 = gdf[gdf['year'] == 2000]
json_2000 = gdf_2000.to_json()

# convert the GeoJson string to an actual python dict 
# (plotly requires the GeoJSON data in python to be an actual dictionary)
# use the built-in json library to convert it
json_2000 = json.loads(json_2000)

# generate an interactive choropleth map of the data for a single year

'''
fig = px.choropleth(
    data_frame = gdf,               # use the year 2000 data
    geojson=json_2000,    
    locations='country',                # name of the dataframe column that contains country names
    color='monthly_anomaly',                    # name of the dataframe column that contains numerical data you want to display
    locationmode='country names',    # leave this as default
    scope='world',                   # change this to world, usa, ... 
    color_continuous_scale="thermal",
    range_color=(0.1, 1.1),
    color_continuous_midpoint=0.5
)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
'''
#fig.show()

#fig.write_html("2000_map.html", include_plotlyjs='cdn')
